In [1]:
import os
import pathlib
import torch
import numpy as np
from torch.nn import functional as F
from torch import nn
import logging
import datetime
import sys
import visdom

In [34]:
def get_positive_and_negative_mask(similarity_matrix, cur_batch_size, positive_nums=3):
    '''
    similarity_matrix:当前batch内所有样本的表示之间的相似度矩阵，shape=[b*b]，与mask序列无关, 32*32
    cur_batch_size: 当前batch内所包含的样本数，绝对样本数，与mask序列的个数无关, 32
    '''
    diag = np.eye(cur_batch_size)
    mask = torch.from_numpy(diag)
    mask = mask.type(torch.bool)  # 单位阵，用于mask掉序列自身与自身的相似度

    oral_batch_size = cur_batch_size // (positive_nums + 1)   # 32//4 = 8

    positives_mask = np.zeros(similarity_matrix.size())  # 32*32的全0矩阵
    for i in range(positive_nums + 1):  # 0 1 2 3
        # k用于控制1的位置，k为正整数，则1的位置向右上对角处移动k个单位，反之则向左下角移动k个单位
        ll = np.eye(cur_batch_size, cur_batch_size, k=oral_batch_size * i)
        lr = np.eye(cur_batch_size, cur_batch_size, k=-oral_batch_size * i)
        positives_mask += ll
        positives_mask += lr
    # 中对角线为2，右上和左下的第8， 16， 24的对角线位置为1

    positives_mask = torch.from_numpy(positives_mask).to(similarity_matrix.device)
    positives_mask[mask] = 0  # 对角线置为0， 右上和左下的第8， 16， 24的对角线位置为1

    negatives_mask = 1 - positives_mask  # 右上和左下的第8， 16， 24的对角线位置为0，其余位置为1
    negatives_mask[mask] = 0  # 对角线位置置0

    return positives_mask.type(torch.bool), negatives_mask.type(torch.bool)

# Mask-使用状态转换机制进行mask，不针对time，针对ravel后的每个feature

In [90]:
def geom_noise_mask_single(L, lm, masking_ratio):
    """
    Randomly create a boolean mask of length `L`, consisting of subsequences of average length lm, masking with 0s a `masking_ratio`
    proportion of the sequence L. The length of masking subsequences and intervals follow a geometric distribution.
    Args:
        L: length of mask and sequence to be masked
        lm: average length of masking subsequences (streaks of 0s)
        masking_ratio: proportion of L to be masked
    Returns:
        (L,) boolean numpy array intended to mask ('drop') with 0s a sequence of length L
    """
    # 并不是mask每个timestep而是mask每个timestep的每个特征，因此可以更好的保留
    keep_mask = np.ones(L, dtype=int)
    # mask ratio: 0.5 -> p_m=1/3, p_u=1/3
    # mask ratio: 0.25 -> p_m=1/3, p_u=1/9
    # mask ratio: 0.75 -> p_m=1/3, p_u=1  被mask的概率最大，每个mask序列之间仅间隔1个unmask的位置
    # lm: 每次产生的连续的mask序列的平均长度
    p_m = 1 / lm  # probability of each masking sequence stopping. parameter of geometric distribution.
    p_u = p_m * masking_ratio / (
            1 - masking_ratio)  # probability of each unmasked sequence stopping. parameter of geometric distribution.
    p = [p_m, p_u]

    # Start in state 0 with masking_ratio probability
    state = int(np.random.rand() > masking_ratio)  # state 0 means masking, 1 means not masking
    for i in range(L):
        keep_mask[i] = state  # here it happens that state and masking value corresponding to state are identical
        if np.random.rand() < p[state]:
            state = 1 - state

    return keep_mask

# FuncTest

In [2]:
torch.randn(2, 5)

tensor([[ 0.0059, -0.3411, -0.5214,  1.9569,  0.3225],
        [-0.9424, -0.0469,  0.5886,  0.7259, -1.1834]])